## Classification Model for identifying Maternal Health Risk

# Introduction

Maternal health remains a critical issue worldwide, especially in rural regions and among lower-middle-class families in emerging countries. The lack of access to proper healthcare, inadequate information about maternal care, and insufficient monitoring during pregnancy contribute to high maternal mortality rates. The significance of timely interventions and constant monitoring during pregnancy cannot be overstated, as each moment is crucial to ensuring the health and safety of both the mother and the baby.
This report investigates maternal health risks using exploratory data analysis and classification techniques such as Logistic regression, SVC and Naive Bayes to identify key factors that contribute to complications during pregnancy. 

The primary question addressed in this project is: What are the key indicators that predict maternal health risks during pregnancy?

To answer this question, a dataset containing information on various maternal health factors was used. Leading to the goal of the project which is to create a predictive model that can evaluate the risk factors associated with pregnancy.

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (train_test_split,
RandomizedSearchCV,
cross_validate,
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from scipy.stats import loguniform, uniform

## About Data 

Data was taken from the UC Irvine Machine Learning Repository \
Dataset link - https://archive.ics.uci.edu/dataset/863/maternal+health+risk \
Column descriptions: 

- Age: Age in years when a woman is pregnant.
- SystolicBP: Upper value of Blood Pressure in mmHg, another significant attribute during pregnancy.
- DiastolicBP: Lower value of Blood Pressure in mmHg, another significant attribute during pregnancy.
- BS: Blood glucose levels is in terms of a molar concentration, mmol/L.
- HeartRate: A normal resting heart rate in beats per minute.
- Risk Level: Predicted Risk Intensity Level during pregnancy considering the previous attribute.

In [2]:
df = pd.read_csv('../data/Maternal Health Risk Data Set.csv')
df.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          1014 non-null   int64  
 1   SystolicBP   1014 non-null   int64  
 2   DiastolicBP  1014 non-null   int64  
 3   BS           1014 non-null   float64
 4   BodyTemp     1014 non-null   float64
 5   HeartRate    1014 non-null   int64  
 6   RiskLevel    1014 non-null   object 
dtypes: float64(2), int64(4), object(1)
memory usage: 55.6+ KB


## EDA

## Modelling

## Hyperparameter Tuning 

# Gaussian Bayes Modeling

In [4]:
train_df, test_df =train_test_split(df, test_size=0.2, random_state=123)

In [5]:
X_train = train_df.drop(columns=["RiskLevel"])
X_test = test_df.drop(columns=["RiskLevel"])
y_train = train_df["RiskLevel"]
y_test = test_df["RiskLevel"]
X_train

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
375,60,120,80,7.8,98.0,75
841,39,110,70,7.9,98.0,80
803,19,120,80,7.0,98.0,70
316,22,120,60,15.0,98.0,80
527,19,120,80,7.0,98.0,70
...,...,...,...,...,...,...
988,25,120,90,12.0,101.0,80
322,65,90,60,6.9,98.0,70
382,17,90,65,7.8,103.0,67
365,22,120,90,7.8,98.0,82


In [7]:
gaus_nb_pipe = make_pipeline(StandardScaler(), GaussianNB())
gaus_nb_pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gaussiannb', GaussianNB())])

In [10]:
cv_results = cross_validate(gaus_nb_pipe, X_train, y_train, return_train_score=True)
results_df = pd.DataFrame(cv_results)
results_df

,fit_time,score_time,test_score,train_score
0,0.010004,0.003999,0.582822,0.604938
1,0.006002,0.003975,0.617284,0.611710
2,0.012978,0.004995,0.604938,0.602465
3,0.006003,0.002998,0.629630,0.596302
4,0.008018,0.001972,0.567901,0.604006


The results of the Gaussian Naive Bayes model are better than the dummy model but not by a lot. Based on this we're going to focus on other models and choose one that performs better.